In [10]:
import gensim.downloader as api
model = api.load('word2vec-google-news-300')

In [9]:
import pandas as pd
import nltk
from gensim.similarities import WmdSimilarity
import pickle

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\young\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords
from nltk import download
stop_words = stopwords.words('english')
import gensim

In [12]:
from nltk import word_tokenize
nltk.download('punkt')  # Download data for tokenizer.

def preprocess(doc):
    doc = doc.lower()  # Lower the text.
    doc = word_tokenize(doc)  # Split into words.
    doc = [w for w in doc if not w in stop_words]  # Remove stopwords.
    doc = [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    return doc

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\young\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
from pyemd import emd

In [14]:
data = pd.read_csv('websites/ParsedWebsitesCleanSample.csv')

In [7]:
# data = data.dropna()

In [15]:
documents = data.values

In [16]:
wmd_corpus = data['text'].apply(preprocess).values

AttributeError: 'float' object has no attribute 'lower'

In [10]:
# instance = WmdSimilarity(wmd_corpus, model, num_best=10)

In [11]:
# pickle.dump(instance,open('WordMoverDistance-Instance.pkl','wb'))

In [18]:
instance = pickle.load(open('WordMoverDistance-WebInstanceSLIM.pkl','rb'))

In [19]:
import re

In [20]:
def regex_text(val):
    if val:
        reg = re.compile(r'\b\w{3,20}\b')
        temp = reg.findall(val)
        temp_set = set(temp)
        temp_list = list(temp_set)
        if len(temp_list) < 20:
            temp2 = temp_list
        elif len(temp_list) < 50:
            temp2 = temp_list[10:]
        elif len(temp_list) < 100:
            temp2 = temp_list[30:]
        else:
            temp2 = temp_list[50:100]
        result = " ".join(temp2) 
        return ("..." + result + "....")
    else:
        return "Empty string found"

In [108]:
def similarityQuery(description):  
    description = description.replace('data', '')
    query = preprocess(description)
    sims = instance[query]  # A query is simply a "look-up" in the similarity class.

    # Print the query and the retrieved documents, together with their similarities.
    results = []
#     print(description)
    for i in range(3):

        row = []
        try:
            row.append(documents[sims[i][0]][0]) # WebsiteKey
            row.append(documents[sims[i][0]][2]) # Website
            row.append(documents[sims[i][0]][3]) # Colors
            row.append(documents[sims[i][0]][4]) # Fonts
            row.append(documents[sims[i][0]][5]) # Text
            row.append(sims[i][1])    # Similarity
            results.append(row)
        except IndexError:
            row.append("sampleWebsiteKey") # WebsiteKey
            row.append("sampleWebsite") # Website
            row.append([['#FFFFFF', 1],['#FFFFFF', 1],['#FFFFFF', 1]]) # Colors
            row.append([['font-family:Arial', '1'],['font-family:Arial', '1'],['font-family:Arial', '1']]) # Fonts
            row.append("sampleText") # Text
            row.append("0.0")    # Similarity
            results.append(row)
            break
        
    resultsDF = pd.DataFrame(results)
    #resultsDF.to_csv(description + '.csv')

    resultsDF.columns = ['WebsiteKey','Website','Colours','Fonts','Text','Similarity']
    resultsDF['Fonts'] = resultsDF['Fonts'].replace({'\" \'': '\", \'', "\' \'": "\', \'", '(\r\n)': ','}, regex=True)
    resultsDF['Colours'] = resultsDF['Colours'].replace({' ': ', '}, regex=True)
    resultsDF['Text'] = resultsDF['Text'].apply(regex_text)
    resultsDF = resultsDF.sort_values(by=['Similarity'],ascending=False)
    return resultsDF#.style.set_properties(subset=['Text'], **{'width': '350px'})

In [91]:
# similarityQuery('I will talk about some cool topics in programming')
similarityQuery('dsfsaweas')

,WebsiteKey,Website,Colours,Fonts,Text,Similarity
0,sampleWebsiteKey,sampleWebsite,"[[#FFFFFF, 0], [#FFFFFF, 0], [#FFFFFF, 0]]","[[font-family:Arial, 0], [font-family:Arial, 0...",...sampleText....,0.0


In [109]:
import ast
def generateResult(description):
    df = similarityQuery(description)
    dict = {}
    flag_f = False
    try:
        aF = [ast.literal_eval(df.iloc[0]['Fonts']), ast.literal_eval(df.iloc[1]['Fonts']), ast.literal_eval(df.iloc[2]['Fonts'])]
        aC = [ast.literal_eval(df.iloc[0]['Colours']), ast.literal_eval(df.iloc[1]['Colours']), ast.literal_eval(df.iloc[2]['Colours'])]
        aT = [df.iloc[0]['Text'], df.iloc[1]['Text'], df.iloc[2]['Text']]
    except:
        aF = [df.iloc[0]['Fonts']]
        aC = [df.iloc[0]['Colours']]
        aT = [df.iloc[0]['Text']] 
        flag_f = True
    
    font_size = []
    for i in range(len(aF)):
        font_size.append(len(aF[i]))
        for j in range(len(aF[i])):
            aF[i][j][1] = int(aF[i][j][1])
    
    color_size = []
    for i in range(len(aC)):
        color_size.append(len(aC[i]))
        for j in range(len(aC[i])):
            if (' ' in aC[i][j][0]) or (',' in aC[i][j][0]):
                aC[i][j][0] = aC[i][j][0].replace(' ','')
                aC[i][j][0] = aC[i][j][0].replace(',','')
    
    if flag_f:
        font_size = [0]
        color_size = [0]
        
    dict['total'] = range(len(aC))
    dict['f_size'] = font_size
    dict['c_size'] = color_size
    dict['fonts'] = aF
    dict['colors'] = aC
    dict['texts'] = aT
    return dict

In [111]:
generateResult("university")

{'total': range(0, 3),
 'f_size': [3, 3, 3],
 'c_size': [3, 3, 3],
 'fonts': [[["font-family:'fl-icons'", 4],
   ["font-family:'object-fit: cover", 3],
   ['font-family:"fl-icons" !important', 2]],
  [['font-family:Arial', 9],
   ['font-family: "revicons"', 8],
   ['font-family: FontAwesome', 5]],
  [['font-family:"Barlow Condensed",arial', 9],
   ['font-family:"Lato",arial', 7],
   ['font-family:arial', 4]]],
 'colors': [[['#446084', 15], ['#f1f1f1', 6], ['#d26e4b', 2]],
  [['#337ab7', 7], ['#f5f5f5', 6], ['#1980B6', 6]],
  [['#f9f9f9', 2], ['#5B5B5', 1], ['#e6e6e6', 1]]],
 'texts': ['...strain current Hot new Account banner Sale right last well taken saying col column submit level wholesale made best strong produce Tour gallery every image little season are few before when dysfunction wondering transparent living place our used Contact receive should Cart favored intercourse Border slider mushroom syringes very visible....',
  '...modal fight VIDEO Story Video image Add pull plus sha